#### MESHpy Test

##### Google Colab execution
Using this Chrome extension, the github-hosted jupyter notebooks may be opened directly in Google Colaboratory
https://chrome.google.com/webstore/detail/open-in-colab/iogfkhleblhcpcekbiedikdehleodpjo
more info here:
https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb

The following two StackOverflow posts helped with managing the dependencies
https://stackoverflow.com/questions/53581278/test-if-notebook-is-running-on-google-colab
https://stackoverflow.com/questions/53793731/using-custom-packages-on-google-colaboratory

[![Open This Notebook In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NOAA-OWP/t-route/blob/master/notebooks/MESHpyReach.ipynb)

In [ ]:
import sys
import subprocess

try:
    import google.colab

    ENV_IS_CL = True
    subprocess.run(["git", "clone", "https://github.com/NOAA-OWP/t-route.git"])
    sys.path.append("/content/src/python_routing_v00")
except:
    ENV_IS_CL = False
    sys.path.append(r"../src/python_routing_v00")


###### Note
9 Sept 2016
The plotting functions seem to require that the cell be executed twice before the output will appear in a standard Jupyter Notebook. 

In [ ]:
from __future__ import division
import pickle
from MESHpyReach import MESHpyReach
import matplotlib.pyplot as plt
from math import ceil
import pandas as pd
import os


def main():
    input_type = "file"
    input_vars = {}
    input_vars["filetype"] = "mesh.py"
    if ENV_IS_CL:
        root = "/content/wrf_hydro_nwm_public/trunk/NDHMS/dynamic_channel_routing/"
    elif not ENV_IS_CL:
        root = os.path.dirname(os.path.abspath(""))
    test_folder = os.path.join(root, r"test")
    output_folder = os.path.join(test_folder, r"output")
    #     input_path = os.path.join(test_folder, r'input', r'text', r'input.txt')
    input_path = os.path.join(test_folder, r"input", r"text", r"input_simple.txt")
    output_path = os.path.join(output_folder, r"out", r"out.txt")

    input_vars[r"input_path"] = input_path

    # reach = DummyReach()
    # reach = SimpleFlowTrace() #DongHa's method.
    # reach = SteadyReach(input_type = input_type, input_vars = input_vars)
    reach = MESHpyReach(input_type=input_type, input_vars=input_vars)
    # reach = MuskCReach()
    # reach = MESHKDDReach()

    reach.compute_initial_state()
    reach.compute_time_steps()

    cols_for_subplots = 4
    fig, axes = plt.subplots(
        nrows=ceil(len(reach.sections) / cols_for_subplots),
        ncols=cols_for_subplots,
        squeeze=False,
    )

    # TODO: make plotting another method within the reach
    for i, section in enumerate(reach.sections):
        """Section Hydrograph Plots"""
        a = pd.Series(time_step.depth for i, time_step in enumerate(section.time_steps))
        # print(m, n)
        m = i // cols_for_subplots
        n = i % cols_for_subplots
        a.plot(ax=axes[m, n])

    cols_for_subplots = 4
    fig2, axes2 = plt.subplots(
        nrows=ceil(len(reach.sections[0].time_steps) / cols_for_subplots),
        ncols=cols_for_subplots,
        squeeze=False,
    )

    # TODO: make plotting another method within the reach
    for j, time_step in enumerate(reach.sections[0].time_steps):
        """Profile Plots"""
        b = pd.Series(
            section.bottom_z + section.time_steps[j].depth
            for i, section in enumerate(reach.sections)
        )
        # print(m, n)
        m = j // cols_for_subplots
        n = j % cols_for_subplots
        # Thanks to this SO post for the easy indexing solution:
        # https://stackoverflow.com/questions/5494974/convert-1d-array-index-to-2d-array-index
        b.plot(ax=axes2[m, n])

    cols_for_subplots = 4
    fig3, axes3 = plt.subplots(
        nrows=ceil(len(reach.sections[0].time_steps) / cols_for_subplots),
        ncols=cols_for_subplots,
        squeeze=False,
    )

    # TODO: make plotting another method within the reach
    for j, time_step in enumerate(reach.sections[0].time_steps):
        """Profile Plots"""
        c = pd.Series(
            section.time_steps[j].depth for i, section in enumerate(reach.sections)
        )
        # print(m, n)
        m = j // cols_for_subplots
        n = j % cols_for_subplots
        c.plot(ax=axes3[m, n])

    # TODO: Use this generator to create a get_depths method of some kind within the Reach class.
    #      Such a method could replace the a = pd.Series... and b = pd.Series... lines in the code above.
    elevations = [
        [section.bottom_z + section.time_steps[j].depth for section in reach.sections]
        for j, _ in enumerate(reach.sections[0].time_steps)
    ]
    depths = [
        [section.time_steps[j].depth for section in reach.sections]
        for j, _ in enumerate(reach.sections[0].time_steps)
    ]
    # print(depths)

    return reach


if __name__ == "__main__":
    reach = main()
    pickle.dumps(reach.sections)
